For this week’s assignment, you are required to investigate the accuracy-computation time tradeoffs of the different optimization algorithms (solvers) that are available for fitting linear regression models in Scikit-Learn. Using the code shared via the Python notebook (part of this week’s uploads archive) where the use of logistic regression was demonstrated, complete the following operations:

1.Among the different classification models included in the Python notebook, which model had the best overall performance? Support your response by referencing appropriate evidence.

In [33]:
from IPython.display import display

In [35]:
%run Machine_Learning_Review_Classification.ipynb

3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)]
Pandas version: 2.2.2
Matplotlib version: 3.8.4
Numpy version: 1.26.4
SciKitLearn version: 1.4.2
My working directory:
C:\Users\rathn
My new working directory:
C:\Users\rathn
              precision    recall  f1-score   support

         0.0       0.76      0.85      0.81       970
         1.0       0.66      0.52      0.58       530

    accuracy                           0.73      1500
   macro avg       0.71      0.68      0.69      1500
weighted avg       0.73      0.73      0.72      1500


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 

Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
              precision    recall  f1-score   support

         0.0       0.76      0.85      0.80       970
         1.0

In [45]:
#results 
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7333   0.718 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 
Logistic_SL1_C_auto    0.7307   0.714 
RandomForest_noCV      0.9993   0.684 
RandomForest_CV        0.9993   0.7   
RandomForest_CV2       0.796    0.716 


--->Logistic_L1_C_10 has the highest overall performance, taking into account both train and test accuracies. It obtained the greatest test accuracy (0.7347), indicating good generalization. RandomForest_noCV got the best train accuracy (0.9993), but its test accuracy (0.684) suggests overfitting. Logistic_L1_C_1 achieves an excellent balance of performance on both training and testing sets. Thus, Logistic_L1_C_1 is the optimal model for generalization.


2.Next, fit a series of logistic regression models, without regularization. Each model should use the same set of predictors (all of the relevant predictors in the dataset) and should use the entire dataset, rather than a fraction of it. Use a randomly chosen 80% proportion of observations for training and the remaining for checking the generalizable performance (i.e., performance on the holdout subset). Be sure to ensure that the training and holdout subsets are identical across all models. Each model should choose a different solver.

In [54]:
pat_df = df_patient.copy()
pat_df["Gender"] = pat_df["Gender"].map({'male': 0, 'female': 1})
pat_df["Race"] = pat_df["Race"].map({'white':0, 'hispanic':1, 'black':2, 'other':3})

#Patient df
X = pat_df[list(vars_left)]
y= pat_df['mortality']

# train and test 
X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(y), random_state=20, test_size=0.20)

In [60]:
X_train.shape

(16000, 25)

In [63]:
X_test.shape

(4000, 25)

In [67]:
y_train.shape

(16000,)

In [69]:
y_test.shape

(4000,)

In [73]:
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#  storing  results
result_scores = {}

# logistic regression model with a specific solver
def create_linear_model(solver_name):
    model = LogisticRegression(
        fit_intercept=True,
        penalty=None,
        solver=solver_name,
        max_iter=10000
    )

    start_tim = time.time()
    model.fit(X_train, y_train)
    end_tim = time.time()

    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))
    elapsed_time = end_tim - start_tim

    result_scores[solver_name] = (train_accuracy, test_accuracy, elapsed_time)


In [95]:
# results
def print_results():
    header = f"\n{'Solver':20} {'Training Subset Accuracy':25} {'Test Subset Accuracy':25} {'Time (s)':10}"
    print(header)
    print("-" * len(header))
    for solver, (train_acc, test_acc, duration) in result_scores.items():
        print(f"{solver:20} {train_acc:<25.4f} {test_acc:<25.4f} {duration:<10.4f}")

# different solvers
solvers = ["lbfgs", "newton-cg", "newton-cholesky", "sag", "saga"]
for solver in solvers:
    create_linear_model(solver)

print_results()


Solver               Training Subset Accuracy  Test Subset Accuracy      Time (s)  
------------------------------------------------------------------------------------
lbfgs                0.7486                    0.7348                    0.2699    
newton-cg            0.7487                    0.7355                    0.1404    
newton-cholesky      0.7487                    0.7355                    0.0668    
sag                  0.7488                    0.7358                    1.2940    
saga                 0.7488                    0.7358                    2.1957    


3.Compare the results of the models in terms of their accuracy (use this as the performance metric to assess generalizability error on the holdout subset) and the time taken (use appropriate timing function). Summarize your results via a table with the following structure:

Solver used                    Training subset accuracy               Holdout subset accuracy                         Time taken

In [98]:
#results based on accuracy.
print_results()


Solver               Training Subset Accuracy  Test Subset Accuracy      Time (s)  
------------------------------------------------------------------------------------
lbfgs                0.7486                    0.7348                    0.2699    
newton-cg            0.7487                    0.7355                    0.1404    
newton-cholesky      0.7487                    0.7355                    0.0668    
sag                  0.7488                    0.7358                    1.2940    
saga                 0.7488                    0.7358                    2.1957    


Accuracy: All solutions perform similarly, with training accuracies of approximately 0.748 and holdout accuracies ranging from 0.7348 to 0.7358. The differences are minimal, implying that all solvers generalize similarly on the holdout subgroup.

Time taken: The Newton-Cholesky solver takes the shortest time (0.0668s), while the Saga solver takes the longest (2.1957s), which is much slower.

The Newton-Cholesky solver strikes a reasonable balance between high accuracy and fast execution time, whereas Saga takes the longest time without considerably enhancing accuracy.

4.Based on the results, which solver yielded the best results? Explain the basis for ranking the models - did you use training subset accuracy? Holdout subset accuracy? Time of execution? All three? Some combination of the three?

Newton-Cholesky is the best solver since it has the highest holdout subset accuracy (0.7355) and the fastest execution time (0.0668s). It strikes the ideal balance between precision and efficiency.

Runner-up: Newton-cg had the same holdout subset accuracy (0.7355) as Newton-cholesky, although it took slightly longer (0.1404s) to execute. As a result, while it achieved the same level of accuracy, it was significantly less efficient.

Other Solvers: lbfgs, sag, and saga achieved comparable holdout accuracies, although their execution times differed. Saga had the slowest execution time (2.1957s) with only a minor improvement in accuracy, hence it is ranked last.

CONCLUSION:-
The ranking relied primarily on holdout subset accuracy, but execution time was also an important factor in the final selection. Newton-Cholesky provided the finest mix of high accuracy and low execution time, making it the clear option.